In [1]:
import pandas as pd
import numpy as np
import scipy as sc
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import model_selection
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler as scaler
from sklearn.metrics import mean_absolute_error as mae
from itertools import product
import datetime
import os
import warnings
warnings.simplefilter(action='ignore')

import lightgbm as lgb

import hyperopt.hp as hp
from hyperopt import fmin, tpe, STATUS_OK, Trials

Предзагрузим необходимые данные:

In [3]:
nonzero_reg=pd.read_csv("data/nonzero_regions.csv").iloc[:,1]

df = pd.read_csv('data/main_data_june.csv')
df.index=df.time
df.drop('time', axis=1, inplace=True)

In [5]:
df.shape

(4368, 102)

In [5]:
predict_dict = {}
for entry in os.listdir('models_predict'):
    for file in os.listdir('models_predict/'+entry):
        reg = file[:4]
        temp = pd.read_csv('models_predict/'+entry+'/'+file).rename(columns={'Unnamed: 0':'time', '0': 'sarma', 'predicted_mean': 'sarma'})
        temp.index=temp.time
        temp.drop('time', axis=1, inplace=True)
        predict_dict[reg] = temp

Формируем датасеты с небольшим количеством новых призанков и добавляем их в словарь.

In [6]:
data_dic = {}
for reg in nonzero_reg:
    # добавить фичи
    
    temp_df = pd.DataFrame({"y_t": df[str(reg)]})
    
    temp_df['reg'] = reg
    
    for i in range(1,13):
        y_t = "y_t_"+str(i)
        temp_df[y_t] = temp_df["y_t"].shift(i)
    
    for i in range(1,3):
        y_t = "y_day_"+str(i)
        temp_df[y_t] = temp_df["y_t"].shift(24*i)
    
    time = pd.to_datetime(temp_df.index)
    
    temp_df['month'] = time.month
    temp_df['day'] = time.day
    temp_df['weekday'] = time.weekday
    temp_df['hour'] = time.hour
    
    temp_df = pd.concat([temp_df, predict_dict[str(reg)]], axis=1)
    
    for i in range(0,6):
        y_="y"+str(i+1)
        temp_df[y_] = temp_df['y_t'].shift(-i-1)

    data_dic[reg] = temp_df.iloc[48:-6]
    

Формируем тренировочные, валидационные и тестовые датасеты.

In [7]:
%%time
X_train, X_val, X_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

for reg in nonzero_reg:
    X_train = pd.concat([X_train, data_dic[reg].loc[data_dic[reg].index < '2016-04-30 23:00:00']])
    X_val = pd.concat([X_val, data_dic[reg].loc[(data_dic[reg].index >= '2016-04-30 23:00:00') & (data_dic[reg].index <= '2016-05-31 17:00:00')]])
    X_test = pd.concat([X_test, data_dic[reg].loc[data_dic[reg].index >= '2016-05-31 23:00:00']])
    
y_train = X_train[['y1', 'y2', 'y3', 'y4', 'y5', 'y6']]
X_train.drop(['y1', 'y2', 'y3', 'y4', 'y5', 'y6'], axis=1, inplace=True)
y_val = X_val[['y1', 'y2', 'y3', 'y4', 'y5', 'y6']]
X_val.drop(['y1', 'y2', 'y3', 'y4', 'y5', 'y6'], axis=1, inplace=True)
y_test = X_test[['y1', 'y2', 'y3', 'y4', 'y5', 'y6']]
X_test.drop(['y1', 'y2', 'y3', 'y4', 'y5', 'y6'], axis=1, inplace=True)

Wall time: 9.65 s


In [6]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(291210, 21) (291210, 6)
(75378, 21) (75378, 6)
(72930, 21) (72930, 6)


In [167]:
X_train.head(5)

,y_t,reg,y_t_1,y_t_2,y_t_3,y_t_4,y_t_5,y_t_6,y_t_7,y_t_8,...,y_t_10,y_t_11,y_t_12,y_day_1,y_day_2,month,day,weekday,hour,sarma
time,,,,,,,,,,,,,,,,,,,,,
2016-01-03 00:00:00,51.0,1075,48.0,53.0,59.0,47.0,54.0,78.0,120.0,142.0,...,105.0,163.0,133.0,24.0,80.0,1,3,6,0,42.987883
2016-01-03 01:00:00,21.0,1075,51.0,48.0,53.0,59.0,47.0,54.0,78.0,120.0,...,125.0,105.0,163.0,9.0,91.0,1,3,6,1,22.461648
2016-01-03 02:00:00,10.0,1075,21.0,51.0,48.0,53.0,59.0,47.0,54.0,78.0,...,142.0,125.0,105.0,11.0,90.0,1,3,6,2,7.118372
2016-01-03 03:00:00,8.0,1075,10.0,21.0,51.0,48.0,53.0,59.0,47.0,54.0,...,120.0,142.0,125.0,5.0,32.0,1,3,6,3,4.413838
2016-01-03 04:00:00,8.0,1075,8.0,10.0,21.0,51.0,48.0,53.0,59.0,47.0,...,78.0,120.0,142.0,6.0,24.0,1,3,6,4,-0.667620


Обучим модель из модуля lightgbm со стандартными параметрами и выведем значение метрики на валидационном датасете:

In [70]:
%%time
lgb_model_1 = lgb.LGBMRegressor(random_state=12)
lgb_model_1.fit(X_train, y_train['y1'])
pred_1_lgb = lgb_model_1.predict(X_val)
print(mae(y_val['y1'], pred_1_lgb))

18.371228410176638
Wall time: 997 ms


Попробуем настроить гиперпараметры с помощью модуля hyperopt. 

In [53]:
space={'max_depth': hp.choice('max_depth',
            [8, 9, 10]),
       'num_leaves': hp.quniform("num_leaves", 255, 1023, 128),
       'reg_alpha' : hp.quniform('reg_alpha', 50,120,5),
       'learning_rate' : hp.choice('learning_rate', [0.01, 0.05, 0.1]),
       'min_data_in_leaf': hp.quniform("min_data_in_leaf", 20, 80, 10),
       'n_estimators': hp.quniform("n_estimators", 100, 550, 50)
    }

In [90]:
def objective(space):
    
    params = {
    "objective": "regression",
    "metric": "mae",
    "boosting": 'gbdt',
    "max_depth": space['max_depth'],  
    "num_leaves": int(space['num_leaves']),
    "n_estimators": int(space['n_estimators']),
    "learning_rate": space['learning_rate'],
    "min_data_in_leaf": int(space['min_data_in_leaf']),
    'reg_alpha': int(space['min_data_in_leaf']),
    "boost_from_average": "false",
    "bagging_seed": 12
}
    train_data = lgb.Dataset(X_train, label=y_train['y3'], categorical_feature=['reg', 'weekday'], free_raw_data=False)
    valid_data = lgb.Dataset(X_val, label=y_val['y3'], categorical_feature=['reg', 'weekday'], free_raw_data=False, reference=train_data)
    
    reg = lgb.train(params,
                    train_data,
                    valid_sets=[valid_data],
                    valid_names=['train', 'test'],
                    num_boost_round=200,
                    early_stopping_rounds=10,
                    verbose_eval=False)
    
    pred = reg.predict(X_val, num_iteration=reg.best_iteration)
    mae_pred = mae(y_val['y1'],pred)
#     print('md:{}\tlr:{}\tleavs:{}\ttest:{}\tmd:{}\treg:{}'.format(params['max_depth'],params['learning_rate'],params['num_leaves'], params['n_estimators'],
#                                                                                   params['min_data_in_leaf'],params['reg_alpha']))
    print ("SCORE:", mae_pred)
    return {'loss': mae_pred, 'status': STATUS_OK}

In [ ]:
%%time
trials = Trials()
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 200,
                        trials = trials)

In [55]:
print(best_hyperparams) #y1 14.839389075928946

{'colsample_bytree': 0.9600407678024615, 'learning_rate': 2, 'max_depth': 2, 'min_data_in_leaf': 20.0, 'n_estimators': 275.0, 'num_leaves': 512.0, 'reg_alpha': 65.0, 'subsample': 0.7460813223006195}


## Парметры подбирались для каждой из 6 моделей. По заданию следующей недели предстоит переобучать модели, добирая новые признаки, заново подбирать параметры модели, поэтому грубо подберем параметры, которые показывают неплохое качество. На валидационном и на тестовом датасете метрики улучшились, немного лучше стала ситуация с метрикой по сравнению с прошлой неделей. Придобавлении дополнительных признаков качество может улучшиться сильнее.

In [11]:
params = {'colsample_bytree': 0.9,
             'learning_rate': 0.1,
             'max_depth': 10,
             'n_estimators': 500,
             'num_leaves': 1023,
             'reg_alpha': 105}

Завернем построение 6 моделей в единную функцию:

In [8]:
def lgb_reg_main(data, params):
    X_train, X_val, X_test, y_train, y_val, y_test = data[0], data[1], data[2], data[3], data[4], data[5]
    y_list = ['y1', 'y2', 'y3', 'y4', 'y5', 'y6']
    
    may_pred, june_pred = pd.DataFrame([]), pd.DataFrame([])
    
    for y in y_list:
        
        train_data = lgb.Dataset(X_train, label=y_train[y], categorical_feature=['reg', 'weekday'], free_raw_data=False)
        valid_data = lgb.Dataset(X_val, label=y_val[y], categorical_feature=['reg', 'weekday'], free_raw_data=False, reference=train_data)
        
        reg = lgb.train(params, train_data, num_boost_round=200, verbose_eval=False)
        
        may_pred[y] = reg.predict(X_val, num_iteration=reg.best_iteration)
        june_pred[y] = reg.predict(X_test, num_iteration=reg.best_iteration)
        
        print(y, '\tmay: ', mae(y_val[y], may_pred[y]),'\tjune: ', mae(y_test[y], june_pred[y]))
        
    return may_pred, june_pred

In [12]:
%%time
may, june = lgb_reg_main((X_train, X_val, X_test, y_train, y_val, y_test), params)

y1 	may:  14.76065476966099 	june:  15.667617271227474
y2 	may:  17.474766909020246 	june:  18.42574613591572
y3 	may:  18.761661034979536 	june:  19.816054417686086
y4 	may:  19.448108500940517 	june:  20.439915402991858
y5 	may:  20.080313891873793 	june:  21.10107872726783
y6 	may:  20.261467918422014 	june:  21.352765237411234
Wall time: 1min 6s


In [30]:
may.head()

,y1,y2,y3,y4,y5,y6
0,67.459085,36.355163,19.878530,13.611407,17.821228,8.426156
1,47.509419,25.361982,18.817343,4.977813,1.293931,9.554429
2,26.322448,13.937088,6.633723,6.294877,9.297331,16.115791
3,10.362531,4.484950,8.398418,8.500697,5.249967,24.048778
4,10.599710,6.040945,14.703171,8.471062,21.403463,38.718616


In [22]:
sum1 = np.sum(np.abs(y_val['y1'].values - may['y1'].values))
sum2 = np.sum(np.abs(y_val['y2'].values - may['y2'].values))
sum3 = np.sum(np.abs(y_val['y3'].values - may['y3'].values))
sum4 = np.sum(np.abs(y_val['y4'].values - may['y4'].values))
sum5 = np.sum(np.abs(y_val['y5'].values - may['y5'].values))
sum6 = np.sum(np.abs(y_val['y6'].values - may['y6'].values))

In [23]:
Q_may = (sum1+sum2+sum3+sum4+sum5+sum6)/(6*739*102)

In [24]:
Q_may

18.464495504149514

In [26]:
sum1 = np.sum(np.abs(y_test['y1'].values - june['y1'].values))
sum2 = np.sum(np.abs(y_test['y2'].values - june['y2'].values))
sum3 = np.sum(np.abs(y_test['y3'].values - june['y3'].values))
sum4 = np.sum(np.abs(y_test['y4'].values - june['y4'].values))
sum5 = np.sum(np.abs(y_test['y5'].values - june['y5'].values))
sum6 = np.sum(np.abs(y_test['y6'].values - june['y6'].values))

In [27]:
Q_june = (sum1+sum2+sum3+sum4+sum5+sum6)/(6*715*102)

In [28]:
Q_june

19.467196198750035

In [164]:
june.head()

,y1,y2,y3,y4,y5,y6
0,28.549688,14.576881,12.296896,5.262460,1.749279,10.766794
1,13.776458,6.612480,3.806291,1.197096,8.035851,24.401427
2,6.309530,2.882693,1.728813,5.854059,24.905855,46.499077
3,4.026374,3.416346,7.450983,17.491150,58.489699,81.503200
4,3.271062,8.307930,21.992971,48.073687,80.868579,71.356664


По строкам у нас расположены концы истории, а по столбцам значения 6 моделей. Таким образом, чтобы получить значения в необходимо виде для сабмишена, нужно просто развернуть датасет в вектор.

In [165]:
june.to_numpy().reshape(june.shape[0]*june.shape[1])

array([28.54968794, 14.57688075, 12.29689566, ..., 29.83118168,
       42.8449099 , 33.0657082 ])

Используя значения индекса и номера региона из X_test, запишем колонку id.

In [131]:
temp = X_test[['reg']]
temp['time'] = temp.index
temp['id'] = temp['reg'].astype(str) + '_' + temp['time'].str[:10]+'_'+temp['time'].str[11:13]
ids = temp['id'].to_list()

In [149]:
fin_id = []
for i in ids:
    new_id = i[:16]
    if i[16:].startswith('0'):
        _ = i[17:]
    else:
        _ = i[16:]
    new_id = new_id + _
    
    for j in range(1, 7):
        fin_id.append(new_id+'_'+str(j))

In [169]:
fin_id[:7]

['1075_2016-05-31_23_1',
 '1075_2016-05-31_23_2',
 '1075_2016-05-31_23_3',
 '1075_2016-05-31_23_4',
 '1075_2016-05-31_23_5',
 '1075_2016-05-31_23_6',
 '1075_2016-06-01_0_1']

In [158]:
len(fin_id)

437580

In [162]:
submissions1 = pd.DataFrame({'id': fin_id, 'y': june.to_numpy().reshape(june.shape[0]*june.shape[1])})

In [163]:
submissions1

,id,y
0,1075_2016-05-31_23_1,28.549688
1,1075_2016-05-31_23_2,14.576881
2,1075_2016-05-31_23_3,12.296896
3,1075_2016-05-31_23_4,5.262460
4,1075_2016-05-31_23_5,1.749279
...,...,...
437575,2168_2016-06-30_17_2,34.163737
437576,2168_2016-06-30_17_3,48.087244
437577,2168_2016-06-30_17_4,29.831182
437578,2168_2016-06-30_17_5,42.844910


In [166]:
submissions1.to_csv('submissions1.csv', index=False)

In [43]:
june

,y1,y2,y3,y4,y5,y6,y1_t,y2_t,y3_t,y4_t,y5_t,y6_t
0,28.549688,14.576881,12.296896,5.262460,1.749279,10.766794,26.0,14.0,5.0,2.0,1.0,7.0
1,13.776458,6.612480,3.806291,1.197096,8.035851,24.401427,14.0,5.0,2.0,1.0,7.0,23.0
2,6.309530,2.882693,1.728813,5.854059,24.905855,46.499077,5.0,2.0,1.0,7.0,23.0,34.0
3,4.026374,3.416346,7.450983,17.491150,58.489699,81.503200,2.0,1.0,7.0,23.0,34.0,72.0
4,3.271062,8.307930,21.992971,48.073687,80.868579,71.356664,1.0,7.0,23.0,34.0,72.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...
72925,20.048231,23.870242,28.634994,19.293549,36.202549,59.082417,0.0,0.0,2.0,1.0,1.0,1.0
72926,15.855099,18.839941,32.464762,11.874835,43.812283,66.966259,0.0,2.0,1.0,1.0,1.0,0.0
72927,14.459717,22.779398,36.860066,24.997923,45.751712,53.613946,2.0,1.0,1.0,1.0,0.0,1.0
72928,18.284217,30.799952,46.939870,25.268603,40.472120,53.917737,1.0,1.0,1.0,0.0,1.0,0.0


In [44]:
june['reg'] = X_test['reg'].values

In [46]:
june.to_csv('data_june.csv', index=False)

### X_test

In [50]:
june.index = X_test.index

In [52]:
june.to_csv('data_june.csv', index=True)